# * Interim : 2024 Target Sales DTAC (Re-org 65 -> 64 Province)
    Jan - May 2024 (K.Voraphon)
    begin Jun 2024 (K.Wanicha)

## Parameter

In [16]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2025, 2, 17)

## ETL Process...

### Step 1 : Import Data Source

In [17]:
''' Rawdata '''

# Target_Sales_DTAC_Y2024
src_file = '../../data/interim/Target_Sales_DTAC_Y2024.xlsx'
src_sheet = 'Rawdata'
src_df = pd.read_excel(src_file, sheet_name=src_sheet, index_col=None) 
src_df.rename(columns={'AREA_CD': 'AREA_KEY', 'DATA_MONTH': 'MONTH_SHORT', 'DATA_YEAR': 'TM_KEY_YR', 'METRIC_VALUES': 'MTH_VALUE'}, inplace=True)
src_df['MONTH_SHORT'] = src_df['MONTH_SHORT'].apply(lambda x: x.upper())
# src_df = src_df.replace(np.nan, None)

print(f'\nsrc_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')
src_df.tail(3)


src_df : 55692 rows, 15 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),AREA_TYPE(TMP),AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
55689,20241218,DEC,2024,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,906111,TRANG,H,HH,906111,TRANG,ALL
55690,20241218,DEC,2024,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,906112,YALA,H,HH,906112,YALA,ALL
55691,20241218,DEC,2024,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,NaN,True corp,H,True corp,True corp,True corp,ALL


In [18]:
''' Master Data '''

# DIM_TIME
dt_file = '../CFW/data/dim_time.csv'
dt_cols = ['TM_KEY_YR', 'MONTH_SHORT', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'DAYS_IN_MONTH']
dt_df = pd.read_csv(dt_file, usecols=dt_cols)
dt_df['MONTH_SHORT'] = dt_df['MONTH_SHORT'].apply(lambda x: x.upper())
# dt_df['MONTH_KEY'] = dt_df['MONTH_SHORT'].str.upper()
# dt_df.tail(3)

# DIM_MOOC_AREA
mooc_file = '../CFW/data/dim_mooc_area.csv'
mooc_cols = ['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_R', 'TDS_RGM_CODE', 'ORGID_H', 'HOP_HINT', 'TDS_PROVINCE', 'PROVINCE_ENG', 'PROVINCE_TH', 'ORGID_HH', 'D_CLUSTER', 'CCAATT', 'REMARK']
mooc_df = pd.read_csv(mooc_file, usecols=mooc_cols)
mooc_df = mooc_df.loc[(mooc_df['REMARK']!='Dummy') & (mooc_df['HOP_HINT']!='True Corp')]
# mooc_df.tail(3)

# Create H level
mooc_h_df = mooc_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT']].drop_duplicates()
mooc_h_df.dropna(how='all', inplace=True)
mooc_h_df['AREA_KEY'] = mooc_h_df['ORGID_H'].apply(lambda x: x if re.search(r'[a-z]', x, re.I) else int(x))
# mooc_h_df.tail(3)

# Create HH level
mooc_hh_df = mooc_df[['ZONE_TYPE', 'ORGID_G', 'TDS_SGMD', 'ORGID_H', 'HOP_HINT', 'ORGID_HH', 'D_CLUSTER']].drop_duplicates()
mooc_hh_df.dropna(how='all', inplace=True)
mooc_hh_df['AREA_KEY'] = mooc_hh_df['ORGID_HH'].astype(int).astype(str)
mooc_hh_df.tail(3)

,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,AREA_KEY
2475,UPC,GX8,Regional Management 8 (South),93X,"TRANG, SATUN, PHATTHALUNG",906108.0,SATUN,906108
2476,UPC,GX7,Regional Management 7 (Central West),74Z,SAMUT SAKHON,902043.0,SAMUT SAKHON,902043
3044,UPC,GX7,Regional Management 7 (Central West),70X,"RATCHABURI, SAMUT SONGKHRAM",902044.0,SAMUT SONGKHRAM,902044


In [19]:
''' Example DataFrame '''

# src_df.tail(3)
# dt_df.tail(3)
# mooc_df.tail(3)
# mooc_h_df.tail(3)
# mooc_h_df.loc[mooc_h_df['ORGID_H'].str.contains('^0')].tail(3)

src_df.tail(3)

# tmp_df = src_df.groupby(['VERSION', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']).agg({'MTH_VALUE': 'sum', 'AREA_TYPE': 'nunique', 'AREA_KEY': 'nunique'}).reset_index()
# tmp_df

,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),AREA_TYPE(TMP),AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
55689,20241218,DEC,2024,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,906111,TRANG,H,HH,906111,TRANG,ALL
55690,20241218,DEC,2024,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,906112,YALA,H,HH,906112,YALA,ALL
55691,20241218,DEC,2024,DB2S020101,Postpaid Gross Adds (Exclude M2M) B2B : DTAC,0.0,DTAC,T,NaN,True corp,H,True corp,True corp,True corp,ALL


### Step 2 : Aggregate Data

In [20]:
'''
    DB1R000900	Prepaid Inflow M1 : DTAC
    DB1S000101	Prepaid Gross Adds : DTAC
    DB2R000500	Postpaid Inflow M1 : DTAC
    DB2S000100	Postpaid Gross Adds : DTAC
'''

''' Filter '''
raw_df = src_df
# raw_df = raw_df.loc[(raw_df['METRIC_CD'].str.contains('^DB1R000900|^DB1S000101|^DB2R000500|^DB2S000100')) & (raw_df['METRIC_CD'].str.contains('[0-9]$|[0-9]A[A-K]$'))]
raw_df = raw_df.loc[raw_df['METRIC_CD'].isin(['DB1R000900', 'DB1S000101', 'DB2R000500', 'DB2S000100'])]

''' Data Test '''
# raw_df = raw_df[raw_df['METRIC_CD'].str.contains('DB1R000900|DB1S000101|DB2R000500|DB2S000100')]
# raw_df = raw_df.loc[raw_df['MONTH_SHORT']=='JAN']
# raw_df = raw_df.loc[raw_df['METRIC_CD']=='DB1S000101']
# raw_df = raw_df.loc[raw_df['AREA_KEY']==80]
# raw_df = raw_df.loc[raw_df['AREA_KEY'].isna()]
raw_df = raw_df.reset_index(drop=True)

print(f'\nraw_df : {raw_df.shape[0]} rows, {raw_df.shape[1]} columns')
raw_df.tail(3)


raw_df : 4284 rows, 15 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,MTH_VALUE,COMP_CD,VERSION,AREA_CD(TMP),AREA_DESC(TMP),AREA_TYPE(TMP),AREA_TYPE,AREA_KEY,AREA_DESC,CHANNEL_CD
4281,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,111.725038,DTAC,T,906111,TRANG,H,HH,906111,TRANG,ALL
4282,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,35.679917,DTAC,T,906112,YALA,H,HH,906112,YALA,ALL
4283,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,34088.816920,DTAC,T,NaN,True corp,H,True corp,True corp,True corp,ALL


In [21]:
''' Re-Org (016, 040) to 080 '''

raw_df = raw_df.groupby(['DATA_DATE', 'MONTH_SHORT', 'TM_KEY_YR', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_TYPE', 'AREA_KEY', 'AREA_DESC']).agg({'MTH_VALUE': 'sum'}).reset_index()
# raw_df = raw_df.loc[raw_df['AREA_TYPE']=='True corp']
raw_df.tail(3)

,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,MTH_VALUE
4269,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,HH,910097,UBON RATCHATHANI,638.792107
4270,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,HH,910098,YASOTHON,4.309551
4271,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,True corp,True corp,True corp,34088.816920


In [22]:
''' Join Area '''

merge_corp_df = raw_df.loc[raw_df['AREA_TYPE']=='True corp']
# merge_corp_df.tail(3)

merge_h_df = pd.merge(raw_df.loc[raw_df['AREA_TYPE']=='H'], mooc_h_df, how='left', on='AREA_KEY')
# merge_h_df = merge_h_df.loc[merge_h_df['AREA_KEY']==80]
# merge_h_df.tail(3)

merge_hh_df = pd.merge(raw_df.loc[raw_df['AREA_TYPE']=='HH'], mooc_hh_df, how='left', on='AREA_KEY')
# merge_hh_df.tail(3)

merge_area_df = pd.concat([merge_corp_df, merge_h_df, merge_hh_df])
print(f'\nprep_agg_df : {merge_area_df.shape[0]} rows, {merge_area_df.shape[1]} columns')
merge_area_df.tail(3)


prep_agg_df : 4272 rows, 18 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,MTH_VALUE,ZONE_TYPE,ORGID_G,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER
3453,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,HH,910096,SURIN,125.857199,UPC,GX6,Regional Management 6 (Northeast 2),33X,"SURIN, SI SA KET",910096.0,SURIN
3454,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,HH,910097,UBON RATCHATHANI,638.792107,UPC,GX6,Regional Management 6 (Northeast 2),34Z,UBON RATCHATHANI,910097.0,UBON RATCHATHANI
3455,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,HH,910098,YASOTHON,4.309551,UPC,GX6,Regional Management 6 (Northeast 2),37X,"YASOTHON, AMNAT CHAROEN",910098.0,YASOTHON


In [23]:
''' Join Period '''

merge_period_df = pd.merge(merge_area_df, dt_df, how='left', on=['TM_KEY_YR', 'MONTH_SHORT'])
# merge_period_df.tail(3)

# Prep Aggregate
prep_agg_df = merge_period_df
prep_agg_df['DAY_VALUE'] = prep_agg_df['MTH_VALUE'] / prep_agg_df['DAYS_IN_MONTH']
print(f'\nprep_agg_df : {prep_agg_df.shape[0]} rows, {prep_agg_df.shape[1]} columns')
prep_agg_df.tail(3)


prep_agg_df : 130360 rows, 23 columns


,DATA_DATE,MONTH_SHORT,TM_KEY_YR,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_TYPE,AREA_KEY,AREA_DESC,...,TDS_SGMD,ORGID_H,HOP_HINT,ORGID_HH,D_CLUSTER,TM_KEY_DAY,DAYS_IN_MONTH,TRUE_TM_KEY_WK,TM_KEY_MTH,DAY_VALUE
130357,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,HH,910098,YASOTHON,...,Regional Management 6 (Northeast 2),37X,"YASOTHON, AMNAT CHAROEN",910098.0,YASOTHON,20241229,31,2024052,202412,0.139018
130358,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,HH,910098,YASOTHON,...,Regional Management 6 (Northeast 2),37X,"YASOTHON, AMNAT CHAROEN",910098.0,YASOTHON,20241230,31,2025001,202412,0.139018
130359,20241218,DEC,2024,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,HH,910098,YASOTHON,...,Regional Management 6 (Northeast 2),37X,"YASOTHON, AMNAT CHAROEN",910098.0,YASOTHON,20241231,31,2025001,202412,0.139018


In [15]:
# agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE'] # , 'FREQUENCY', 'REMARK'

# # P : Nationwide
# agg_p_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
# agg_p_df['AREA_NO'] = 1
# agg_p_df['AREA_TYPE'] = 'P'
# agg_p_df['AREA_CD'] = 'P'
# agg_p_df['AREA_NAME'] = 'Nationwide'
# agg_p_df = agg_p_df.loc[:, agg_cols]
# agg_p_df[agg_p_df['TM_KEY_DAY']==20240501]

,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,DAY_VALUE,MTH_VALUE
484,2024,202405,2024018,20240501,DB1R000900,Prepaid Inflow M1 : DTAC,DTAC,T,1,P,P,Nationwide,8.574852e+06,2.658204e+08
485,2024,202405,2024018,20240501,DB1S000101,Prepaid Gross Adds : DTAC,DTAC,T,1,P,P,Nationwide,4.720787e+04,1.463444e+06
486,2024,202405,2024018,20240501,DB2R000500,Postpaid Inflow M1 : DTAC,DTAC,T,1,P,P,Nationwide,1.117542e+06,3.464380e+07
487,2024,202405,2024018,20240501,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,1,P,P,Nationwide,2.506581e+03,7.770400e+04


In [24]:
''' Aggregate P, G, H, HH '''

agg_cols = ['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'AREA_NO', 'AREA_TYPE', 'AREA_CD', 'AREA_NAME', 'DAY_VALUE', 'MTH_VALUE'] # , 'FREQUENCY', 'REMARK'

# P : Nationwide
agg_p_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_p_df['AREA_NO'] = 1
agg_p_df['AREA_TYPE'] = 'P'
agg_p_df['AREA_CD'] = 'P'
agg_p_df['AREA_NAME'] = 'Nationwide'
agg_p_df = agg_p_df.loc[:, agg_cols]
# agg_p_df[agg_p_df['TM_KEY_DAY']==20240501]

# G : Region
agg_g_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_G', 'TDS_SGMD']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_g_df['AREA_NO'] = 2
agg_g_df['AREA_TYPE'] = 'G'
agg_g_df.rename(columns={'ORGID_G': 'AREA_CD'}, inplace=True)
agg_g_df.rename(columns={'TDS_SGMD': 'AREA_NAME'}, inplace=True)
agg_g_df = agg_g_df.loc[:, agg_cols]
# agg_g_df[agg_g_df['TM_KEY_DAY']==20240501]

# H : HOP_HINT
agg_h_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_H', 'HOP_HINT']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_h_df['AREA_NO'] = 3
agg_h_df['AREA_TYPE'] = 'H'
agg_h_df.rename(columns={'ORGID_H': 'AREA_CD'}, inplace=True)
agg_h_df.rename(columns={'HOP_HINT': 'AREA_NAME'}, inplace=True)
agg_h_df = agg_h_df.loc[:, agg_cols]
# agg_h_df[agg_h_df['TM_KEY_DAY']==20240501]

# HH : D_CLUSTER
agg_hh_df = prep_agg_df.groupby(['TM_KEY_YR', 'TM_KEY_MTH', 'TRUE_TM_KEY_WK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'COMP_CD', 'VERSION', 'ORGID_HH', 'D_CLUSTER']).agg({'MTH_VALUE': 'sum', 'DAY_VALUE': 'sum'}).reset_index()
agg_hh_df['AREA_NO'] = 4
agg_hh_df['AREA_TYPE'] = 'HH'
agg_hh_df['ORGID_HH'] = agg_hh_df['ORGID_HH'].astype(int).astype(str)
agg_hh_df.rename(columns={'ORGID_HH': 'AREA_CD'}, inplace=True)
agg_hh_df.rename(columns={'D_CLUSTER': 'AREA_NAME'}, inplace=True)
agg_hh_df = agg_hh_df.loc[:, agg_cols]
# agg_hh_df[agg_hh_df['TM_KEY_DAY']==20240601]

# Concat DataFrame
last_agg_all_df = pd.concat([agg_p_df, agg_g_df, agg_h_df, agg_hh_df], ignore_index=True)
last_agg_all_df['AGG_TYPE'] = 'S'
last_agg_all_df['FREQUENCY'] = 'Daily'
# last_agg_all_df['REMARK'] = 'Sales target DTAC Y2024(Jan-May) : as of 23-May-24(K.Voraphon)'
# last_agg_all_df['REMARK'] = last_agg_all_df['TM_KEY_MTH'].apply(lambda x: 'Sales target DTAC Y2024(Jan-May) : as of 23-May-24(K.Voraphon)' if x>=202401 and x<=202412 else 'Ref : K.Wanicha')
last_agg_all_df['REMARK'] = last_agg_all_df['TM_KEY_MTH'].apply(lambda x: 'H Level 64 Province' if x>=202401 and x<=202403 else 'HH Level 96 Cluster')

print(f'\nlast_agg_all_df : {last_agg_all_df.shape[0]} rows, {last_agg_all_df.shape[1]} columns')
# last_agg_all_df.loc[last_agg_all_df['TM_KEY_DAY']==20240601]
last_agg_all_df.tail(3)


last_agg_all_df : 212472 rows, 17 columns


,TM_KEY_YR,TM_KEY_MTH,TRUE_TM_KEY_WK,TM_KEY_DAY,METRIC_CD,METRIC_NAME,COMP_CD,VERSION,AREA_NO,AREA_TYPE,AREA_CD,AREA_NAME,DAY_VALUE,MTH_VALUE,AGG_TYPE,FREQUENCY,REMARK
212469,2024,202412,2025001,20241231,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,4,HH,910096,SURIN,4.059910,125.857199,S,Daily,HH Level 96 Cluster
212470,2024,202412,2025001,20241231,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,4,HH,910097,UBON RATCHATHANI,20.606197,638.792107,S,Daily,HH Level 96 Cluster
212471,2024,202412,2025001,20241231,DB2S000100,Postpaid Gross Adds : DTAC,DTAC,T,4,HH,910098,YASOTHON,0.139018,4.309551,S,Daily,HH Level 96 Cluster


### Step 3 : Insert to "INTERIM_VINSIGHT_DATA"
    Delete -> Insert

In [25]:
''' Input Parameter '''

# Create list
month_list = last_agg_all_df['TM_KEY_MTH'].drop_duplicates().tolist()
mt_cd_list = last_agg_all_df['METRIC_CD'].drop_duplicates().tolist()

if len(mt_cd_list) == 1:
    mt_cd_list = str(mt_cd_list).replace(r'[', '(').replace(r']', ')')
else:
    mt_cd_list = tuple(mt_cd_list)

# Create Param
# v_param = dict(mth_start=202406, mth_end=202408, metric_cd=mt_cd_list)
v_param = dict(mth_start=min(month_list), mth_end=max(month_list), metric_cd=mt_cd_list)
v_target_schema = 'AUTOKPI'
v_target_table = 'INTERIM_VINSIGHT_DATA'

# query_delete = f"DELETE {v_target_schema}.{v_target_table} WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} AND METRIC_CD IN {v_param['metric_cd']}"
query_delete = f"""
    DELETE {v_target_schema}.{v_target_table} 
    WHERE TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']} 
    AND METRIC_CD IN {v_param['metric_cd']}
"""

print(f"\nParameter...\n\n   -> TM_KEY_MTH BETWEEN {v_param['mth_start']} AND {v_param['mth_end']}\n   -> METRIC_CD IN {v_param['metric_cd']}")
print(f'\nDataFrame...\n\n   -> last_agg_all_df : {last_agg_all_df.shape[0]} rows, {last_agg_all_df.shape[1]} columns') 
print(f'\nquery_delete...\n{query_delete}')


Parameter...

   -> TM_KEY_MTH BETWEEN 202401 AND 202412
   -> METRIC_CD IN ('DB1R000900', 'DB1S000101', 'DB2R000500', 'DB2S000100')

DataFrame...

   -> last_agg_all_df : 212472 rows, 17 columns

query_delete...

    DELETE AUTOKPI.INTERIM_VINSIGHT_DATA 
    WHERE TM_KEY_MTH BETWEEN 202401 AND 202412 
    AND METRIC_CD IN ('DB1R000900', 'DB1S000101', 'DB2R000500', 'DB2S000100')



In [26]:
''' Load Data '''

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')


# Create rows from DataFrame
rows = [tuple(x) for x in last_agg_all_df.values]


# Connect : AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected')
cur = conn.cursor()
print(f'\nProcessing...')


try:
    # # Truncate
    # cur.execute(f"TRUNCATE TABLE {v_target_schema}.{v_target_table}")
    # print(f'\n   -> TRUNCATE : "{v_target_table}" : Done !')

    # Delete
    cur.execute(query_delete)
    print(f'\n   -> DELETE : "{v_target_table}" : Done !')
    
    # Insert
    cur.executemany(f"""
        INSERT INTO {v_target_table} 
        (TM_KEY_YR, TM_KEY_MTH, TRUE_TM_KEY_WK, TM_KEY_DAY, METRIC_CD, METRIC_NAME, COMP_CD, VERSION, AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME, DAY_VALUE, MTH_VALUE, AGG_TYPE, FREQUENCY, REMARK) 
        VALUES (:1,:2,:3,:4,:5,:6,:7,:8,:9,:10,:11,:12,:13,:14,:15,:16,:17)
        """, rows)
    print(f'\n   -> INSERT : "{v_target_table}" : Done !')

    cur.close()
    conn.commit()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')
    print(f'\nJob Done !!!')



Job Start... 2025-02-17, 19:21:27

AKPIPRD : Connected

Processing...

   -> DELETE : "INTERIM_VINSIGHT_DATA" : Done !

   -> INSERT : "INTERIM_VINSIGHT_DATA" : Done !

AKPIPRD : Disconnected

Job Done !!!
